In [1]:
train_batch_folder = "data"
train_batches_json = ["20210511_seg-batch-37-2021-05-11.json", "20210511_seg-batch-38-2021-05-11.json", "20210511_seg-batch-38-2021-05-11.json", 
                      "20210511_seg-batch-39-2021-05-11.json", "20210511_seg-batch-40-2021-05-11.json", "20210511_seg-batch-41-2021-05-11.json",
                      "20210511_seg-batch-42-2021-05-11.json", "20210511_seg-batch-43-2021-05-11.json", "20210511_seg-batch-44-2021-05-11.json",
                      "20210511_seg-batch-45-2021-05-11.json", "20210511_seg-batch-46-2021-05-11.json", "20210511_seg-batch-47-2021-05-11.json",
                      "20210511_seg-batch-48-2021-05-11.json", "20210511_seg-batch-49-2021-05-11.json", "20210511_seg-batch-50-2021-05-11.json",
                      "20210511_seg-batch-51-2021-05-11.json", "20210511_seg-batch-52-2021-05-11.json", "20210511_seg-batch-53-2021-05-11.json",
                      "20210511_seg-batch-54-2021-05-11.json", "20210511_seg-batch-55-2021-05-11.json", "20210511_seg-batch-56-2021-05-11.json",
                      "20210511_seg-batch-57-2021-05-11.json", "20210511_seg-batch-58-2021-05-11.json", "20210511_seg-batch-59-2021-05-11.json",
                      "20210511_seg-batch-60-2021-05-11.json", "20210511_seg-batch-61-2021-05-11.json", "20210511_seg-batch-62-2021-05-11.json",
                      "20210511_seg-batch-62-2021-05-11.json", "20210511_seg-batch-63-2021-05-11.json", "20210511_seg-batch-64-2021-05-11.json",
                      "20210511_seg-batch-65-2021-05-11.json", "20210511_seg-batch-66-2021-05-11.json", "20210511_seg-batch-67-2021-05-11.json",
                    ]
valid_batch_folder = "data"
valid_batches_json = ["20210511_seg-batch-68-2021-05-11.json", "20210511_seg-batch-69-2021-05-11.json", "20210511_seg-batch-70-2021-05-11.json", 
                      "20210511_seg-batch-71-2021-05-11.json", "20210511_seg-batch-72-2021-05-11.json", "20210511_seg-batch-73-2021-05-11.json",
                      "20210511_seg-batch-74-2021-05-11.json",
                    ]
label_dict = {"Road": 1, "BAU":2, "SW": 3}

download_images = False
resize_height_image = 720//3
resize_width_image = 576//3
channels_image = 3

max_epochs = 100
devices = [0]
accelerator = "gpu"
log_every_n_steps = 1
train_batch_size = 32
valid_batch_size = 32

In [4]:
from configs import segmentation



In [2]:
import torch as th
from torch.utils.data import ConcatDataset, DataLoader
from torchvision import transforms
from pytorch_lightning import Trainer
import numpy as np
import albumentations as A
import pprint as pp
import json
import cv2

from datasets.augmentations import ApplyAlbumination, compose, transposeImage, convertImage2float, convertMask2long
from datasets.io import labeler_image_mask_load, rescale_labeler_image_mask_load
from datasets import labeler_download_lot, ImageMaskDataset
from models.unet import UnetTrainer

th.manual_seed(42)

# sanity check for data label dictionary
n_classes = len(list(label_dict.keys())) + 1
assert n_classes == max(list(label_dict.values())) + 1

# data formatting
# data loading from labeler format
image_mask_load_fun=labeler_image_mask_load(maks_label_dict=label_dict)
# data loadinf and rescale
image_load_fun = rescale_labeler_image_mask_load((resize_height_image, resize_width_image), image_mask_load_fun=image_mask_load_fun)

# data augmentation
augmentation = ApplyAlbumination(A.Compose([
    A.RandomCrop(width=128, height=128),
    A.OneOf([
        A.RandomSizedCrop(min_max_height=(50, 101), height=128, width=128, p=0.5),
        A.PadIfNeeded(min_height=128, min_width=128, p=0.5)
    ], p=0.2),    
    A.VerticalFlip(p=0.5),              
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
        A.GridDistortion(p=0.5),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)                  
        ], p=0.8),
    A.CLAHE(p=0.8),
    A.RandomBrightnessContrast(p=0.8),    
    A.RandomGamma(p=0.8)]))
                                 
# data augmentation and formatting
image_mask_train_transform = compose( [ augmentation, transposeImage, convertImage2float, convertMask2long ] )
# data formatting
image_mask_valid_transform = compose( [ transposeImage, convertImage2float, convertMask2long ] )

# load train data batches
train_batches = []
for train_batch_json in train_batches_json:
    image_folder, mask_folder  = labeler_download_lot(os.path.join(train_batch_folder, train_batch_json), only_labels=not download_images)
    ds = ImageMaskDataset(image_folder, mask_folder, image_prefix="", mask_prefix="", image_suffix=".png", mask_suffix=".json",
                          image_mask_load_fun=image_load_fun, image_mask_transform=image_mask_train_transform)
    train_batches += [ds]
train_dataset = ConcatDataset(train_batches)
    
# load valid data batches
valid_batches = []
for valid_batch_json in valid_batches_json:
    image_folder, mask_folder  = labeler_download_lot(os.path.join(valid_batch_folder, valid_batch_json), only_labels=not download_images)
    ds = ImageMaskDataset(image_folder, mask_folder, image_prefix="", mask_prefix="", image_suffix=".png", mask_suffix=".json",
                          image_mask_load_fun=image_load_fun, image_mask_transform=image_mask_valid_transform)
    valid_batches += [ds]
valid_dataset = ConcatDataset(valid_batches)

train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch_size, shuffle=False)

# create model and trainer
model = UnetTrainer(channels_image,n_classes)
trainer = Trainer( max_epochs=max_epochs, accelerator=accelerator, devices=devices, logger=True, log_every_n_steps=log_every_n_steps)
trainer.fit(model=model, train_dataloaders=train_dataloader, val_dataloaders=valid_dataloader)


/home/mviti/miniconda3/envs/Train_py37_th1_13_cu11/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
BufferedDataloader:   0%|          | 0/39 [00:00<?, ?it/s]dataloaders 2022-12-02:13:12:28,304 WARNING  [io.py:63] found annotation label not in label dictionary: Parking
dataloaders 2022-12-02:13:12:28,341 WARNING  [io.py:63] found annotation label not in label dictionary: Parking
BufferedDataloader:  23%|██▎       | 9/39 [00:00<00:00, 81.08it/s]dataloaders 2022-12-02:13:12:28,396 WARNING  [io.py:63] found annotation label not in label dictionary: Parking
dataloaders 2022-12-02:13:12:28,424 WARNING  [io.py:63] found annotation label not in label dictionary: Parking
BufferedDataloader:  46%|████▌     | 18/39 [00:00<00:00, 82.19it/s]dataloaders 2022-12-02:13:12:28,492 WARNING  [io.py:63] found annotation labe

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/mviti/miniconda3/envs/Train_py37_th1_13_cu11/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


/home/mviti/miniconda3/envs/Train_py37_th1_13_cu11/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:241: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Epoch 0: 100%|██████████| 48/48 [00:07<00:00,  6.51it/s, loss=0.773, v_num=2]

...

Epoch 98: 100%|██████████| 48/48 [00:06<00:00,  6.90it/s, loss=0.0975, v_num=2]

dataloaders 2022-12-02:13:28:27,641 DEBUG    [local.py:265] open file: /raid-dgx3/mviti/gits/SegmentationTrainingPipeline/lightning_logs/version_2/checkpoints/epoch=98-step=3861.ckpt


Epoch 99: 100%|██████████| 48/48 [00:07<00:00,  6.85it/s, loss=0.0834, v_num=2]

dataloaders 2022-12-02:13:28:35,766 DEBUG    [local.py:265] open file: /raid-dgx3/mviti/gits/SegmentationTrainingPipeline/lightning_logs/version_2/checkpoints/epoch=99-step=3900.ckpt
`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 48/48 [00:08<00:00,  5.93it/s, loss=0.0834, v_num=2]
